# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [11]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

Using TensorFlow backend.


In [12]:
ttt = pd.read_csv('tic-tac-toe.csv')
ttt = pd.get_dummies(ttt)
ttt

,class,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,...,MR_x,BL_b,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x
0,True,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,1,0,0,1,0
1,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
2,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,1
3,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,1,0,0,1,0,0
4,True,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,False,0,1,0,0,0,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1
954,False,0,1,0,0,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,1
955,False,0,1,0,0,0,1,0,1,0,...,1,0,0,1,0,1,0,0,0,1
956,False,0,1,0,0,0,1,0,1,0,...,1,0,0,1,0,1,0,0,0,1


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [16]:
X_train.shape

(641, 27)

In [90]:
from sklearn.model_selection import train_test_split

X= ttt.iloc[:,1:]
y= ttt.iloc[:,0].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [111]:
# Build the model.
model = Sequential([
  Dense(64, activation='relu', input_shape=(27,)),
  Dense(64, activation='relu'),
  Dense(2, activation='softmax'),   ## Usually the softmax is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  y_train,
  epochs=20,
  batch_size=8,
)

# Evaluate the model.
print(model.evaluate(X_test,y_test))

model.save_weights('tic-tac-toe.model')

Epoch 1/20
641/641 [==============================] - 0s 407us/step - loss: 0.6211 - accuracy: 0.6661
Epoch 2/20
641/641 [==============================] - 0s 335us/step - loss: 0.5249 - accuracy: 0.7270
Epoch 3/20
641/641 [==============================] - 0s 537us/step - loss: 0.4545 - accuracy: 0.7816
Epoch 4/20
641/641 [==============================] - 0s 239us/step - loss: 0.3926 - accuracy: 0.8268
Epoch 5/20
641/641 [==============================] - 0s 195us/step - loss: 0.3302 - accuracy: 0.8736
Epoch 6/20
641/641 [==============================] - 0s 193us/step - loss: 0.2503 - accuracy: 0.9158
Epoch 7/20
641/641 [==============================] - 0s 223us/step - loss: 0.1738 - accuracy: 0.9610
Epoch 8/20
641/641 [==============================] - 0s 220us/step - loss: 0.1202 - accuracy: 0.9797
Epoch 9/20
641/641 [==============================] - 0s 193us/step - loss: 0.0886 - accuracy: 0.9813
Epoch 10/20
641/641 [==============================] - 0s 200us/step - loss: 0.063

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [110]:
X_test.reset_index(inplace = True, drop = True)
y_test = y_test.reset_index(drop = True)

In [109]:
values = np.random.randint(0,316,20)

print('Predicted:', np.argmax(model.predict(X_test.iloc[values]), axis=1))
print('Real:     ', np.array(y_test[values]))

Predicted: [1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1]
Real:      [1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

**Which approach(es) did you find helpful to improve your model performance?**

##### Model already showed low loss and high accuracy ... played around with epochs and batch size...